In [31]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
from glob import glob
from skimage.io import ImageCollection, imsave
from skimage.color import rgb2hsv, rgb2gray, gray2rgb
from skimage.transform import warp, SimilarityTransform, AffineTransform, ProjectiveTransform
from skimage.feature import (ORB, match_descriptors, corner_harris, corner_fast, corner_peaks, BRIEF,
                             plot_matches)
from skimage.measure import ransac
from numpy.random import randint
from skimage.data import imread
from skimage.measure import label

from skimage.graph import route_through_array
from sklearn.feature_extraction import image
from subprocess import Popen, PIPE
from multiprocessing import Pool, freeze_support, cpu_count
import itertools
import math
%matplotlib inline
path_to_enblend = '/Users/jhansen/Applications/enblend-enfuse-4.0-mac/enblend'
tmp_out = '/tmp/out.png'
tmp_base = '/tmp/base.png'
tmp_img = '/tmp/img.png'
data_dir = '../data/jpg/'

img1 = imread('IMG_1765.jpg')[:,:,2]
img2 = imread('IMG_1767.jpg')[:,:,2]

In [2]:
def hsv_imread(img_path):
    return rgb2hsv(imread(img_path))

def gray_imread(img_path):
    return rgb2gray(imread(img_path))

def load_images(search_dir, ftype):
    search_path = os.path.join(search_dir, '*'+ftype)
    files = glob(search_path)
    imgs = []
    for ff in files:
        imgs.append({'name':os.path.split(ff)[1]})
    return imgs

In [3]:
def compare(*images, **kwargs):
    """
    Function to display images side by side (from skimage example)

    Parameters
    ----------
    image0, image1, ....: ndarray
        Images to display
    labels: list
        Labels for the different images
    """
    f, ax = plt.subplots(1, len(images), **kwargs)
    ax = np.array(ax, ndmin=1)

    labels = kwargs.pop('labels', None)
    labels = [''] * len(images)
    for n, (image, label) in enumerate(zip(images, labels)):
        ax[n].imshow(image, interpolation='nearest', cmap=plt.gray())
        ax[n].set_title(label)
        ax[n].axis('off')
    plt.tight_layout()

In [4]:
def get_best_matches(k1, k2, matches):
    src = k1[matches[:,0]][:,::-1]
    dst = k2[matches[:,1]][:,::-1]
    # if there are not enough matches, this fails
    model_robust, inliers = ransac((src, dst), AffineTransform,
                                   min_samples=20, residual_threshold=1,
                                   max_trials=40)

    return model_robust, inliers

def develop_metadata_mosaic():
    # set max pitch and roll angles as qulity
    pass

In [5]:



def plot_two_matches(img1, img2, k1, k2, matches):
    fig, ax = plt.subplots(nrows=1, ncols=2)
    plt.gray()
    #plot_matches(ax, img1, img2, k1, k2, matches)
    ax[0].imshow(img1)
    ax[0].axis('off')
    ax[0].scatter(k1[:, 1], k1[:, 0],  facecolors='none', edgecolors='r')

    ax[1].imshow(img2)
    ax[1].axis('off')
    ax[1].scatter(k2[:, 1], k2[:, 0], facecolors='none', edgecolors='r')
    plt.show()
    
    plt.show()
    
def plot_two_keypoints(img1, img2, k1, k2, s1=1, s2=1):
    fig, ax = plt.subplots(nrows=1, ncols=2)
    plt.gray()

    ax[0].imshow(img1)
    ax[0].axis('off')
    ax[0].scatter(k1[:, 1], k1[:, 0], facecolors='none', edgecolors='r')

    ax[1].imshow(img2)
    ax[1].axis('off')
    ax[1].scatter(k2[:, 1], k2[:, 0], facecolors='none', edgecolors='r')
    plt.show()

In [6]:

def generate_costs(diff_image, mask, vertical=True, gradient_cutoff=2.):
    """
    Ensures equal-cost paths from edges to region of interest.
    
    Parameters
    ----------
    diff_image : ndarray of floats
        Difference of two overlapping images.
    mask : ndarray of bools
        Mask representing the region of interest in ``diff_image``.
    vertical : bool
        Control operation orientation.
    gradient_cutoff : float
        Controls how far out of parallel lines can be to edges before
        correction is terminated. The default (2.) is good for most cases.
        
    Returns
    -------
    costs_arr : ndarray of floats
        Adjusted costs array, ready for use.
    """
    if vertical is not True:
        return tweak_costs(diff_image.T, mask.T, vertical=vertical,
                           gradient_cutoff=gradient_cutoff).T
    
    # Start with a high-cost array of 1's
    costs_arr = np.ones_like(diff_image)
    
    # Obtain extent of overlap
    row, col = mask.nonzero()
    cmin = col.min()
    cmax = col.max()

    # Label discrete regions
    cslice = slice(cmin, cmax + 1)
    labels = label(mask[:, cslice])
    
    # Find distance from edge to region
    upper = (labels == 0).sum(axis=0)
    lower = (labels == 2).sum(axis=0)
    
    # Reject areas of high change
    ugood = np.abs(np.gradient(upper)) < gradient_cutoff
    lgood = np.abs(np.gradient(lower)) < gradient_cutoff
    
    # Give areas slightly farther from edge a cost break
    costs_upper = np.ones_like(upper, dtype=np.float64)
    costs_lower = np.ones_like(lower, dtype=np.float64)
    costs_upper[ugood] = upper.min() / np.maximum(upper[ugood], 1)
    costs_lower[lgood] = lower.min() / np.maximum(lower[lgood], 1)
    
    # Expand from 1d back to 2d
    vdist = mask.shape[0]
    costs_upper = costs_upper[np.newaxis, :].repeat(vdist, axis=0)
    costs_lower = costs_lower[np.newaxis, :].repeat(vdist, axis=0)
    
    # Place these in output array
    costs_arr[:, cslice] = costs_upper * (labels == 0)
    costs_arr[:, cslice] +=  costs_lower * (labels == 2)
    
    # Finally, place the difference image
    costs_arr[mask] = diff_image[mask]
    
    return costs_arr

In [24]:
def calc_enblend(timg, warped):
    pass

def add_alpha_channel(img, background=-1):
    """Add an alpha layer to the image.
    
    The alpha layer is set to 1 for foreground and 0 for background.
    """
    if img.ndim == 2:
        img = gray2rgb(img)
    return np.dstack((img, (img != background)))


In [25]:
def add_alpha(img, mask=None):
    """
    Adds a masked alpha channel to an image.
    
    Parameters
    ----------
    img : (M, N[, 3]) ndarray
        Image data, should be rank-2 or rank-3 with RGB channels
    mask : (M, N[, 3]) ndarray, optional
        Mask to be applied. If None, the alpha channel is added
        with full opacity assumed (1) at all locations.
    """
    if mask is None:
        mask = np.ones_like(img)
        
    if img.ndim == 2:
        img = gray2rgb(img)
    
    return np.dstack((img, mask))

In [26]:
def simple_merge(base_warped, img_warped, base_mask, img_mask):
    
    # Add the three images together. This could create dtype overflows!
    # We know they are are floating point images after warping, so it's OK.
    merged = (base_warped + img_warped)

    # Track the overlap by adding the masks together
    # Multiply by 1.0 for bool -> float conversion
    overlap = (base_mask * 1.0 + img_mask)

    # Normalize through division by `overlap` - but ensure the minimum is 1
    norm = merged / np.maximum(overlap, 1)

    return norm

In [27]:
def detect_and_extract(detector, img):
    detector.detect_and_extract(img)
    keypoints = detector.keypoints
    descriptors = detector.descriptors
    return keypoints, descriptors

In [7]:
def find_output_shape(base_img, model_robust):
    r, c = base_img.shape[:2]
    
    corners = np.array([[0,0], 
                        [0,r],
                        [c,0],
                        [c,r]])
    
    warped_corners = model_robust(corners)
    all_corners = np.vstack((warped_corners, corners))
    # The overally output shape will be max - min
    corner_min = np.min(all_corners, axis=0)
    corner_max = np.max(all_corners, axis=0)
    output_shape = (corner_max - corner_min)
    # Ensure integer shape with np.ceil and dtype conversion
    output_shape = np.ceil(output_shape[::-1]).astype(int)
    return output_shape, corner_min


In [8]:
def find_two_matches(base_img, img, base_k, img_k, base_d, img_d, min_matches=10):
    matches = match_descriptors(base_d, img_d, cross_check=True)
    
    #   * src (image to be registered): pano2
    #   * dst (reference image): pano1, our middle frame registration target
    src = img_k[matches[:,1]][:,::-1]
    dst = base_k[matches[:,0]][:,::-1]
    
    # if there are not enough matches, this fails
    if matches.shape[0] > min_matches:
        model_robust, inliers = ransac((src, dst), ProjectiveTransform,
                                   min_samples=8, residual_threshold=1,
                                   max_trials=600)

        ransac_matches = matches[inliers]
        return model_robust, ransac_matches
    else:
        return np.zeros((0, 2)), np.zeros((0, 2))

In [9]:

num_keypoints = 800
pano_imgs = ImageCollection('*.jpg')
img_col = load_images('../data/jpg/', 'jpg')
img_feat = {}
num_imgs = len(img_col)
min_matches = 40


In [10]:
def remove_empty_edges(img):
    def get_mask(sums):
        if sum(sums) > 0:
            first = sums.index(1)
            last = sums[::-1].index(1)

            num_ones = (len(sums)-first)-last
            out = [0]*first + [1]*num_ones + [0]*last
            return out
        else:
            return sums
    
    #for ax in range(len(img.shape)):
    axes = [0, 1]
    for ax in range(2):
        sums = np.sum(img, axis=axes[ax])
        # make a mask of zero lines in image 
        sums= [bool(x) for x in sums]
        empty = get_mask(list(sums)) 
        img = np.compress(empty, img, axis=axes[ax-1])
    return img


In [29]:
"""Read SIFT and SURF feature files.
See Also
--------
http://people.cs.ubc.ca/~lowe/keypoints/
http://www.vision.ee.ethz.ch/~surf/
"""

__all__ = ['load_sift', 'load_surf']

import numpy as np


def _sift_read(f, mode='SIFT'):
    """Read SIFT or SURF features from a file.
    Parameters
    ----------
    f : string or open file
        Input file generated by the feature detectors from
        http://people.cs.ubc.ca/~lowe/keypoints/ or
        http://www.vision.ee.ethz.ch/~surf/
    Returns
    -------
    data : record array with fields
      - row: int
          row position of feature
      - column: int
          column position of feature
      - scale: float
          feature scale
      - orientation: float
          feature orientation
      - data: array
          feature values
    """
    if not hasattr(f, 'readline'):
        f = file(f, 'r')

    if mode == 'SIFT':
        nr_features, feature_len = map(int, f.readline().split())
        datatype = np.dtype([('row', float), ('column', float),
                             ('scale', float), ('orientation', float),
                             ('data', (float, feature_len))])
    else:
        mode = 'SURF'
        feature_len = int(f.readline()) - 1
        nr_features = int(f.readline())
        datatype = np.dtype([('column', float), ('row', float),
                             ('second_moment', (float, 3)),
                             ('sign', float), ('data', (float, feature_len))])
    data = np.fromfile(f, sep=' ')
    if data.size != nr_features * datatype.itemsize / np.dtype(float).itemsize:
        raise IOError("Invalid %s feature file." % mode)

    return data.view(datatype)


def load_sift(f):
    return _sift_read(f, mode='SIFT')


def load_surf(f):
    return _sift_read(f, mode='SURF')

load_sift.__doc__ = _sift_read.__doc__
load_surf.__doc__ = _sift_read.__doc__

In [12]:
import surf

ImportError: No module named surf

In [13]:
def surf_detect_and_extract(img):
    ip = surf.surf(img)
    #points = surf.interest_points(img, 6, 24, 1, max_points=1024)
    #descs = surf.descriptors(img, points, descriptor_only=True)
    print(ip.shape)
    k = ip[:, :2]
    d = ip[:,6:]
    def rotate(y,x, a):
        sa = np.sin(a)
        ca = np.cos(a)
        return (ca*x-sa*y, sa*x+ca*y)
    return k, d, ip
# not working - descriptor is not correct
k1, d1, ip1 = surf_detect_and_extract(img1)
k2, d2, ip2 = surf_detect_and_extract(img2)
plot_two_keypoints(img1, img2, k1, k2)
f1 = surf.show_surf(img1, ip1)
f2 = surf.show_surf(img2, ip2)
plt.figure()
plt.imshow(f1)
plt.figure()
plt.imshow(f2)

NameError: global name 'surf' is not defined

In [14]:
from skimage.feature import CENSURE
from mahotas.features import zernike_moments

from mahotas.features import surf
import timeit
import time

In [32]:
import skimage.data as data
def z(img1, img2):
    br = zernike()
    #k1 = corner_peaks(corner_harris(img1, method='eps', eps=.001, sigma=3), min_distance=5)
    k1 = corner_peaks(corner_fast(img1, n=4, threshold=.001), min_distance=5)
    br.extract(img1, k1)
    d1 = br.descriptors
    k1 = k1[br.mask]

    #k2 = corner_peaks(corner_harris(img2, method='eps', eps=.001, sigma=3), min_distance=5)
    k2 = corner_fast(img2, n=4, threshold=.001)
    br = zernike()
    br.extract(img2, k2)
    d2 = br.descriptors
    k2 = k2[br.mask]

    matches = match_descriptors(d1, d2, cross_check=True)
    model_robust, ransac_matches = find_two_matches(img1, img2, 
                                                    k1, k2, 
                                                    d1, d2)
    #fig, ax = plt.subplots(nrows=1, ncols=1)
    #plt.gray()
    #plot_matches(ax, img1, img2, k1, k2, ransac_matches)
    prec = round(ransac_matches.shape[0]/float(matches.shape[0]), 2)

    #print('zern keypoints', k1.shape[0], k2.shape[0], 'matches', matches.shape[0],ransac_matches.shape[0], prec )
    return prec
def o(img1, img2):
    # fast_threshold - decide whether pixels are brighter or darker,
    # decrease for more corners
    # harris_k smaller for detection of sharp corners
    orb = ORB(n_keypoints=600, fast_n=5, 
              fast_threshold=0.02, 
              harris_k=.01, n_scales=10)
    # Zernike moments are not a texture feature, but rather a global measure of how the mass is distributed.

    #k1, d1 = detect_and_extract(orb, img1)
    #k2, d2 = detect_and_extract(orb, img2)

    #matches = match_descriptors(d1, d2, cross_check=True)
    #model_robust, ransac_matches = find_two_matches(img1, img2, 
    #                                                k1, k2, 
    #                                                d1, d2)
    #prec = round(ransac_matches.shape[0]/float(matches.shape[0]), 2)
    #print('orb keypoints', k1.shape[0], k2.shape[0], 'matches', matches.shape[0],  ransac_matches.shape[0], prec )
    #fig, ax = plt.subplots(nrows=1, ncols=1)
    #plt.gray()
    #plot_matches(ax, img1, img2, k1, k2, ransac_matches)
    #return prec
a = 10
st = time.time()
for i in range(a):
    z(img1, img2)
print('took', (time.time()-st)/float(a))

ValueError: attempt to get argmin of an empty sequence

In [38]:
def z(img1):
    br = zernike()
    #k1 = corner_peaks(corner_harris(img1, method='eps', eps=.001, sigma=3), min_distance=5)
    k1 = corner_peaks(corner_fast(img1, n=4, threshold=.001), min_distance=5)
    br.extract(img1, k1)
    d1 = br.descriptors
    kk = k1[br.mask]
    print(k1.shape, kk.shape, d1.shape)
z(img1)

((122, 2), (103, 2), (103, 256))


array([[117, 117, 118, ..., 119, 120, 120],
       [118, 118, 118, ..., 117, 120, 119],
       [118, 118, 118, ..., 118, 119, 119],
       ..., 
       [113, 114, 114, ..., 119, 118, 118],
       [113, 114, 115, ..., 119, 118, 117],
       [114, 114, 115, ..., 118, 117, 117]], dtype=uint8)

In [16]:

tform = AffineTransform(scale=(1.2,1.2),translation=(0,-100))
img3 = warp(img2, tform)
img4 = rotate(img2, 25)
img5 = rotate(img3, 25)

poh, pohw, pohr, pohwr = 0,0,0,0
zoh, zohw, zohr, zohwr = 0,0,0,0
a = 1
for x in range(a):
    poh += o(img1, img2)
    pohw += o(img1, img3)
    pohr += o(img1, img4)
    pohwr += o(img1, img5)
for x in range(a):
    zoh += z(img1, img2)
    zohw += z(img1, img3)
    zohr += z(img1, img4)
    zohwr += z(img1, img5)

oa = [poh/float(a), pohw/float(a), pohr/float(a), pohwr/float(a)]
za = [zoh/float(a), zohw/float(a), zohr/float(a), zohwr/float(a)]

#img1 = imread('IMG_1765.jpg')[:,:,2]
#img2 = imread('IMG_1767.jpg')[:,:,2]
img1 = imread('IMG_1771.jpg')[:,:,2]
img2 = imread('IMG_1772.jpg')[:,:,2]
tform = AffineTransform(scale=(1.2,1.2),translation=(0,-100))
img3 = warp(img2, tform)
img4 = rotate(img2, 25)
img5 = rotate(img3, 25)

poh, pohw, pohr, pohwr = 0,0,0,0
zoh, zohw, zohr, zohwr = 0,0,0,0

for x in range(a):
    poh += o(img1, img2)
    pohw += o(img1, img3)
    pohr += o(img1, img4)
    pohwr += o(img1, img5)
for x in range(a):
    zoh += z(img1, img2)
    zohw += z(img1, img3)
    zohr += z(img1, img4)
    zohwr += z(img1, img5)

eoa = [poh/float(a), pohw/float(a), pohr/float(a), pohwr/float(a)]
eza = [zoh/float(a), zohw/float(a), zohr/float(a), zohwr/float(a)]

NameError: name 'rotate' is not defined

In [ ]:
#ORB Easy Images	63.2	51.3	0.567	37.3
#Zernike Easy Images	69.6	59.5	10.6	15.6
#ORB Hard Images	42.5	30.8	47.2	31.4
#Zernike Hard Images	57.3	37.5	7.5	14.3
import pandas as pd
rows=['ORB Easy Images', 'Zernike Easy Images', 'ORB Hard Images', 'Zernike Hard Images', ]
cols= ["Sample Image", "Scaled", "Rotated", "Scaled and Rotated"]
d = np.array([oa, za, eoa, eza])
p = pd.DataFrame(d, index=rows, columns=cols)
p

In [ ]:
p.T.plot()

In [17]:
def _zern_loop(image, descriptors, keypoints, pos0, pos1):
    for p in range(pos0.shape[0]):
        pr0 = pos0[p, 0]
        pc0 = pos0[p, 1]
        pr1 = pos1[p, 0]
        pc1 = pos1[p, 1]
        for k in range(keypoints.shape[0]):
            kr = keypoints[k, 0]
            kc = keypoints[k, 1]
            if image[kr + pr0, kc + pc0] < image[kr + pr1, kc + pc1]:
                descriptors[k, p] = True

In [18]:
from skimage.feature.util import _mask_border_keypoints, DescriptorExtractor
class zernike(DescriptorExtractor):
    
    def __init__(self, descriptor_size=256, patch_size=49,
                  sigma=1, sample_seed=1):
        self.descriptor_size = descriptor_size
        self.patch_size = patch_size
        self.sigma = sigma
        self.sample_seed = sample_seed

        self.descriptors = None
        self.mask = None
    
    def extract(self, image, keypoints):
        patch_size = self.patch_size
        desc_size = self.descriptor_size
        random = np.random.RandomState()
        random.seed(self.sample_seed)
        samples = (patch_size / 5.0) * random.randn(desc_size * 8)
        samples = np.array(samples, dtype=np.int32)
        samples = samples[(samples < (patch_size // 2))
                          & (samples > - (patch_size - 2) // 2)]

        pos1 = samples[:desc_size * 2].reshape(desc_size, 2)
        pos2 = samples[desc_size * 2:desc_size * 4].reshape(desc_size, 2)
        
        pos1 = np.ascontiguousarray(pos1)
        pos2 = np.ascontiguousarray(pos2)
        self.mask = _mask_border_keypoints(image.shape, keypoints,
                                           patch_size // 2)
        keypoints = np.array(keypoints[self.mask, :], dtype=np.intp,
                             order='C', copy=False)
        self.descriptors = np.zeros((keypoints.shape[0], desc_size),
                                    dtype=bool, order='C')
        
        _zern_loop(image, self.descriptors.view(np.uint8), keypoints,
                    pos1, pos2)


In [19]:
def find_all_matches(unmatched, 
                     matched=[], 
                     fail_limit=3,
                     num_keypoints=800,
                     min_to_match=20, 
                     do_plot=False):
    
    num_unmatched = len(unmatched)
    if num_unmatched == 0:
        print("NONE left unmatched")
        return matched
    if num_unmatched == 1:
        print("ONE left unmatched")
        matched.append(unmatched[0])
        return matched
    
    print("=====================================")
    base = unmatched[0]
    if 'img' in base.keys():
        base_img = base['img']
    else:
        base_img = imread(os.path.join(data_dir, base['name']))
        
    orb = ORB(n_keypoints=num_keypoints, fast_threshold=0.05)

    base_unmatched = []
    
    
    base_name = base['name'].split('.')[0]
    base_matched = 0

    # go through each image that is not yet matched
    for xx, timg in enumerate(unmatched[1:]):
        # if the image has not yet been loaded, load it now
        if 'img' not in timg.keys():
            timg['img'] = imread(os.path.join(data_dir, timg['name']))
            
        # for now, only use the 3rd channel
        img = timg['img'][:,:,2]
        
        
        base_k, base_d = detect_and_extract(orb, base_img[:,:,2])
        # if we haven't recorded the keypoints for this image, get them now
        if 'keypoints' in timg.keys():
            img_k = timg['keypoints']
            img_d = timg['descriptors']
        else:
            img_k, img_d = detect_and_extract(orb, img)
            timg['keypoints'] = img_k
            timg['descriptors'] = img_d

        
        matches = match_descriptors(base_d, img_d, cross_check=True)
        
        model_robust, ransac_matches = find_two_matches(base_img[:,:,2], img, 
                                                            base_k, img_k, 
                                                            base_d, img_d)
        if do_plot:
            print('matches', matches.shape[0], ransac_matches.shape[0])
            #plot_two_keypoints(ax, base_img, img, base_k, img_k)
            fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15,12))
            plt.title('%s #### %s' %(base_name, timg['name']))
            plt.gray()

            ax[0].imshow(base_img)
            ax[0].axis('off')
            ax[0].scatter(base_k[:, 1], base_k[:, 0], facecolors='none', edgecolors='r')

            ax[1].imshow(img)
            ax[1].axis('off')
            ax[1].scatter(img_k[:, 1], img_k[:, 0], facecolors='none', edgecolors='r')
            plt.show()
        
        if ransac_matches.shape[0] < min_to_match:
            print("------------", matches.shape[0], "ransac", ransac_matches.shape[0])
            base_unmatched.append(timg)
            if len(base_unmatched) >= fail_limit:
                # add two since we've already added this timg
                base_unmatched.extend(unmatched[xx+2:])
                break
        else:
            #print('ransac matches', ransac_matches.shape)
            base_img = find_mask(base_img, timg['img'], model_robust) 
            base_img = img
            base_matched += 1
            print("***********", base_matched)
            base_name+= '_' + timg['name'].split('.')[0]
            # AFTER an image has been matched, remove from memory
            
            

    # if we were able to match some images to this base_img that
    # were not matched in the last run, call again until 
    # the number of unmatched images stops decreasing
    
    #print('num previous unmatched', num_unmatched)
    #print("could not match %s out of %s imgs" %(len(base_unmatched), 
    #                                            len(unmatched)-1))
    # not_matched must be > 0
    # the new number of matches must be less than last time's not matched
    
    
    if 'base_matched' in base.keys():
        all_base_matched = base['base_matched'] + base_matched
    else:
        all_base_matched = 1 + base_matched
    rr = {'name':base_name, 'img':base_img, 'base_matched':base_matched}
    if (len(base_unmatched)) > 0:
        #if base_matched > 0 :
            #base_unmatched.insert(0, rr)
            #print("!!!!!!!!!!! 1 match %s, unmatch %s" %(len(matched), len(base_unmatched)))
            
            #return find_all_matches(base_unmatched, matched, num_keypoints, min_to_match)
        #else:
        print("DECLARING %s as matched, ending with %s matches" %(base_name, base_matched))
        matched.append(rr)
        #print("!!!!!!!!!!! 2 match %s, unmatch %s" %(len(matched), len(base_unmatched)))
        return find_all_matches(base_unmatched, matched, fail_limit, num_keypoints, min_to_match, do_plot)
    else:
        matched.append(rr)
        #print("!!!!!!!!!!! 3 match %s, unmatch %s" %(len(matched), len(base_unmatched)))
        return matched
    

In [20]:
def minimum_cost_merge(base_warped, img_warped, base_mask, img_mask):
    # Start with the absolute value of the difference image.
    # np.abs is necessary because we don't want negative costs!
    costs = generate_costs(np.abs(img_warped - base_warped),
                           img_mask & base_mask)
    costs[0,  :] = 0
    costs[-1, :] = 0

    output_shape = base_warped.shape
    # Arguments are:
    #   cost array
    #   start pt
    #   end pt
    #   can it traverse diagonally
    ymax = output_shape[1] - 1
    xmax = output_shape[0] - 1

    # Start anywhere along the top and bottom, left of center.
    mask_pts01 = [[0,    ymax // 3],
                  [xmax, ymax // 3]]

    # Start anywhere along the top and bottom, right of center.
    mask_pts12 = [[0,    2*ymax // 3],
                  [xmax, 2*ymax // 3]]
    
    pts, _ = route_through_array(costs, mask_pts01[0], mask_pts01[1], fully_connected=True)

    # Convert list of lists to 2d coordinate array for easier indexing
    pts = np.array(pts)
   
    # Start with an array of zeros and place the path
    _img_mask = np.zeros_like(img_warped, dtype=np.uint8)
    _img_mask[pts[:, 0], pts[:, 1]] = 1


    # Labeling starts with zero at point (0, 0)
    _img_mask[label(_img_mask, connectivity=1) == 0] = 1
    
    _base_mask = ~(_img_mask).astype(bool)

    base_color = gray2rgb(base_warped)
    img_color = gray2rgb(img_warped)
    base_final = add_alpha(base_warped, _base_mask)
    img_final = add_alpha(img_warped, _img_mask)
    
    # Start with empty image
    base_combined = np.zeros_like(base_warped)
    
    base_combined += base_warped * _base_mask
    base_combined += img_warped * _img_mask
    
    return base_combined



   

In [21]:
range(0, 100, 8)

[0, 8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96]

In [22]:
def patchmaker(img, imsize=(255,255), percent_overlap=10):
    """
    Split an image into overlapping patches
       
    Parameters
    ----------
    img : ndarray
        Image from which to extract patches
    imsize : tuple of ints
        size of patches
    percent_overlap : int
        Percent as int of overlap desired between overlapping images
        
    Returns
    -------
    patches : list of imsize overlapping segments of the image
        
    """
    # store the patches here
    patches = []
    patch_rows = imsize[0]
    patch_cols = imsize[1]
    if 0 < percent_overlap < 100:
        # determine how many pixels to overlap
        non_overlap_rows = int(patch_rows*.01*(100-percent_overlap))
        non_overlap_cols = int(patch_cols*.01*(100-percent_overlap))
    else:
        non_overlap_rows = patch_rows
        non_overlap_cols = patch_cols
        
    # row indexes into the original image 
    r1, c1 = 0,0
    # column indexes into the original image
    r2, c2 = imsize
    # while the last index of the patch image is less than the size of the original, keep going
    while r2 < img.shape[0]:
        c1 = 0
        c2 = c1 + patch_cols
        while c2 < img.shape[1]:
            patch = img[r1:r2, c1:c2]
            patches.append(patch)
            c1 += non_overlap_rows
            c2 = c1 + patch_cols 
        r1 += non_overlap_rows
        r2 = r1 + patch_rows
    return patches

In [23]:
def find_mask(base_img, img, model_robust):
    # what type of interpolation
    # 0: nearest-neighbor
    # 1: bi-linear
    warp_order = 1

    output_shape, corner_min = find_output_shape(base_img, model_robust)
    #print("output_shape", output_shape, corner_min)
    #print(model_robust.scale, model_robust.translation, model_robust.rotation)
    
    # This in-plane offset is the only necessary transformation for the base image
    offset = SimilarityTransform(translation= -corner_min)
    base_warped = warp(base_img[:,:,2], offset.inverse, order=warp_order, 
                      output_shape = output_shape, cval=-1)
    base_color = warp(base_img, offset.inverse, order=warp_order, 
                      output_shape = output_shape, cval=-1)   
    # warp image corners to new position in mosaic
    transform = (model_robust + offset).inverse
    
    img_warped = warp(img[:,:,2], transform, order=warp_order, 
                      output_shape=output_shape, cval=-1)
    img_color = warp(img, transform, order=warp_order, 
                      output_shape=output_shape, cval=-1)
    base_mask = (base_warped != -1)
    base_warped[~base_mask] = 0

    img_mask = (img_warped != -1)
    img_warped[~img_mask] = 0

    #convert to rgb
    #base_alpha = add_alpha(base_color, base_mask)
    img_alpha = np.dstack((img_color, img_mask))
    base_alpha = np.dstack((base_color, base_mask))

    plt.imsave(tmp_base, base_alpha )
    plt.imsave(tmp_img, img_alpha )
    cmd = [path_to_enblend, tmp_base, tmp_img, '-o', tmp_out]

    p = Popen(cmd, stdin=PIPE, stdout=PIPE, stderr=PIPE)
    output, err = p.communicate(b"input data that is passed to subprocess' stdin")
    rc = p.returncode
    # remove alpha channel
    
    if os.path.exists(tmp_out):
        out = imread(tmp_out)[:,:,:3]
    else:
        print("couldnt find out image")
        print(rc, output, err)
        plt.figure()
        plt.imshow(base_alpha)
        plt.figure()

        plt.imshow(img_alpha)
        plt.show()
        out = base_alpha[:,:,:3]
    #if you don't have enblend, you can use one of these
    #merged_img = simple_merge(base_warped, img_warped, base_mask, img_mask)
    #merged_img = minimum_cost_merge(base_warped, img_warped, base_mask, img_mask)
    #merged_edges = remove_empty_edges(merged_img)
    return out



In [24]:
def find_alpha(base_img, img, model_robust):
    # what type of interpolation
    # 0: nearest-neighbor
    # 1: bi-linear
    warp_order = 1

    output_shape, corner_min = find_output_shape(base_img, model_robust)
    #print("output_shape", output_shape, corner_min)
    #print(model_robust.scale, model_robust.translation, model_robust.rotation)
    
    # This in-plane offset is the only necessary transformation for the base image
    offset = SimilarityTransform(translation= -corner_min)
    base_warped = warp(base_img[:,:,2], offset.inverse, order=warp_order, 
                      output_shape = output_shape, cval=-1)
    base_color = warp(base_img, offset.inverse, order=warp_order, 
                      output_shape = output_shape, cval=-1)   
    # warp image corners to new position in mosaic
    transform = (model_robust + offset).inverse
    
    #img_warped = warp(img[:,:,2], transform, order=warp_order, 
    #                  output_shape=output_shape, cval=-1)
    img_color = warp(img, transform, order=warp_order, 
                      output_shape=output_shape, cval=-1)
    #base_mask = (base_warped != -1)
    #base_warped[~base_mask] = 0

    img_mask = (img_warped != -1)
    #img_warped[~img_mask] = 0

    #convert to rgb
    #base_alpha = add_alpha(base_color, base_mask)
    img_alpha = np.dstack((img_color, img_mask))
    #base_alpha = np.dstack((base_color, base_mask))

    #plt.imsave(tmp_base, base_alpha )
    #plt.imsave(tmp_img, img_alpha )
    #cmd = [path_to_enblend, tmp_base, tmp_img, '-o', tmp_out]

    #p = Popen(cmd, stdin=PIPE, stdout=PIPE, stderr=PIPE)
    #output, err = p.communicate(b"input data that is passed to subprocess' stdin")
    #rc = p.returncode
    # remove alpha channel
    
    #if os.path.exists(tmp_out):
    #    out = imread(tmp_out)[:,:,:3]
    #else:
    #    print("couldnt find out image")
    #    print(rc, output, err)
    #    plt.figure()
    #    plt.imshow(base_alpha)
    #    plt.figure()#

    #    plt.imshow(img_alpha)
    #    plt.show()
    #    out = base_alpha[:,:,:3]
    #if you don't have enblend, you can use one of these
    #merged_img = simple_merge(base_warped, img_warped, base_mask, img_mask)
    #merged_img = minimum_cost_merge(base_warped, img_warped, base_mask, img_mask)
    #merged_edges = remove_empty_edges(merged_img)
    return tmp_alpha


In [56]:
imsave?

In [57]:
#patches = patchmaker(img_col[0])
if 0:
    unmatched = img_col
    params = [[1, 500, 50], [2, 800, 20], [1000, 1000, 10], [1000, 10000, 7]]
    for param in params:
        matched = find_all_matches(unmatched, [], param[0], param[1], param[2], False)
        print('found %s matches with' %len(matched), param)
    print("HAVE %s MATCHES" %len(matched))

In [58]:
if 0:
    for xx, timg in enumerate(matched):
        plt.figure(figsize=(14,14))
        plt.title("NUM %s NAME %s" %(xx, timg['name']))
        plt.imshow(timg['img'])
        plt.show()


In [43]:


def split_find_all_matches(i):
    """Convert list to arguments for find_all_matches to work with multiprocessing
    pool"""
    return find_all_matches(*i)

#all_matched = find_all_matches(unmatched, [], param[0], param[1], param[2], False)

#pool = Pool(processes=cpu_count())
#split_unmatched = [img_col[:6]]
#params = [[], 1, 500, 50]
#all_matched = pool.map(split_find_all_matches, 
#                       itertools.izip(split_unmatched, 
#                                      itertools.repeat(params)))

           

In [44]:
print("HE")

HE


In [45]:
itertools.izip?

In [46]:
x = [4, 5, 6]
a = {'stuff':2}
b = {'333':2}
z = [a, b]

In [47]:
x.extend(z)
x

[4, 5, 6, {'stuff': 2}, {'333': 2}]